<a href="https://colab.research.google.com/github/rackhamsauer/deep-learning-challenge/blob/main/NeuralNetworkSSR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [66]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [67]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
drop_df = application_df.drop(columns=['EIN', 'NAME'])


In [68]:
# Determine the number of unique values in each column.
unique_counts = drop_df.nunique()
unique_counts

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [69]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts_df = drop_df['APPLICATION_TYPE'].value_counts()
application_type_counts_df

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [70]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 200

application_types_to_replace = drop_df['APPLICATION_TYPE'].value_counts()[drop_df['APPLICATION_TYPE'].value_counts() < cutoff_value].index.tolist()


# Replace in dataframe
for app in application_types_to_replace:
    drop_df['APPLICATION_TYPE'] = drop_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
drop_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [71]:
drop_df.columns

Index(['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS',
       'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')

In [72]:
# Look at CLASSIFICATION value counts for binning
print(drop_df['CLASSIFICATION'].value_counts())


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [73]:
# You may find it helpful to look at CLASSIFICATION value counts >1
filtered_counts = drop_df['CLASSIFICATION'].value_counts()[drop_df['CLASSIFICATION'].value_counts() > 1]

# Print the filtered counts
print(filtered_counts)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64


In [74]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value = 1000

classifications_to_replace = drop_df['CLASSIFICATION'].value_counts()[drop_df['CLASSIFICATION'].value_counts() < cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    drop_df['CLASSIFICATION'] = drop_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
drop_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [79]:
categorical_list = drop_df.select_dtypes(include=['object']).columns.tolist()
categorical_list

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [81]:
# Convert categorical data to numeric with `pd.get_dummies`
drop_df_encoded = pd.get_dummies(drop_df, columns=categorical_list)
print(drop_df_encoded.head())

   STATUS  ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  \
0       1     5000              1                       0   
1       1   108590              1                       0   
2       1     5000              0                       0   
3       1     6692              1                       0   
4       1   142590              1                       0   

   APPLICATION_TYPE_T10  APPLICATION_TYPE_T19  APPLICATION_TYPE_T3  \
0                     1                     0                    0   
1                     0                     0                    1   
2                     0                     0                    0   
3                     0                     0                    1   
4                     0                     0                    1   

   APPLICATION_TYPE_T4  APPLICATION_TYPE_T5  APPLICATION_TYPE_T6  ...  \
0                    0                    0                    0  ...   
1                    0                    0                    0  

In [82]:
# Split our preprocessed data into our features and target arrays
# Features (X)
X = drop_df_encoded.drop(columns=['IS_SUCCESSFUL'])

# Target (y)
y = drop_df_encoded['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [83]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [86]:
print(X_train)

       STATUS  ASK_AMT  APPLICATION_TYPE_Other  APPLICATION_TYPE_T10  \
15029       1  1725423                       0                     0   
22784       1     5000                       0                     0   
13575       1     5000                       0                     0   
22153       1     5000                       0                     0   
17052       1     5000                       0                     0   
...       ...      ...                     ...                   ...   
27743       1     5000                       0                     0   
24452       1     5000                       0                     0   
1404        1     5000                       0                     0   
27605       1     5000                       0                     0   
14571       1   126690                       0                     0   

       APPLICATION_TYPE_T19  APPLICATION_TYPE_T3  APPLICATION_TYPE_T4  \
15029                     0                    1              

In [93]:
# Define the model - deep neural net
input_features = X_train.shape[1]
hidden_node1 = 8
hidden_node2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node1, activation="relu", input_dim=input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 8)                 352       
                                                                 
 dense_13 (Dense)            (None, 5)                 45        
                                                                 
 dense_14 (Dense)            (None, 1)                 6         
                                                                 
Total params: 403 (1.57 KB)
Trainable params: 403 (1.57 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [94]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [95]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.6245 - accuracy: 0.6970
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5863 - accuracy: 0.7250
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5768 - accuracy: 0.7287
Epoch 4/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5707 - accuracy: 0.7287
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5658 - accuracy: 0.7292
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5629 - accuracy: 0.7287
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5612 - accuracy: 0.7302
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5596 - accuracy: 0.7282
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5588 - accuracy: 0.7293
Epoch 10/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5577 - accura

In [96]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5652 - accuracy: 0.7279 - 566ms/epoch - 2ms/step
Loss: 0.5651822090148926, Accuracy: 0.7279300093650818


In [99]:
# Export our model to HDF5 file
file_path = "/Users/staciesauer/GitHub/deep-learning-challenge/model1.h5"

# Export the model to the specified HDF5 file
nn.save(file_path)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [101]:
# Define the model - deep neural net
input_features = X_train.shape[1]
hidden_node1 = 10
hidden_node2 = 6

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node1, activation="relu", input_dim=input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 10)                440       
                                                                 
 dense_16 (Dense)            (None, 6)                 66        
                                                                 
 dense_17 (Dense)            (None, 1)                 7         
                                                                 
Total params: 513 (2.00 KB)
Trainable params: 513 (2.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [102]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [103]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=150)

Epoch 1/150
804/804 [==============================] - 4s 3ms/step - loss: 0.6192 - accuracy: 0.6822
Epoch 2/150
804/804 [==============================] - 2s 3ms/step - loss: 0.5672 - accuracy: 0.7196
Epoch 3/150
804/804 [==============================] - 2s 3ms/step - loss: 0.5597 - accuracy: 0.7233
Epoch 4/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5571 - accuracy: 0.7238
Epoch 5/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5553 - accuracy: 0.7242
Epoch 6/150
804/804 [==============================] - 2s 3ms/step - loss: 0.5540 - accuracy: 0.7249
Epoch 7/150
804/804 [==============================] - 3s 3ms/step - loss: 0.5530 - accuracy: 0.7296
Epoch 8/150
804/804 [==============================] - 2s 3ms/step - loss: 0.5520 - accuracy: 0.7299
Epoch 9/150
804/804 [==============================] - 2s 3ms/step - loss: 0.5514 - accuracy: 0.7306
Epoch 10/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5506 - accura

In [104]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5574 - accuracy: 0.7284 - 1s/epoch - 4ms/step
Loss: 0.5573925375938416, Accuracy: 0.728396475315094


In [105]:
# Define the model - deep neural net
input_features = X_train.shape[1]
hidden_node10 = 15
hidden_node20 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node10, activation="relu", input_dim=input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node20, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 15)                660       
                                                                 
 dense_19 (Dense)            (None, 10)                160       
                                                                 
 dense_20 (Dense)            (None, 1)                 11        
                                                                 
Total params: 831 (3.25 KB)
Trainable params: 831 (3.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [106]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [107]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 6s 4ms/step - loss: 0.5939 - accuracy: 0.7013
Epoch 2/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5595 - accuracy: 0.7269
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5542 - accuracy: 0.7288
Epoch 4/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5523 - accuracy: 0.7297
Epoch 5/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5506 - accuracy: 0.7310
Epoch 6/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5493 - accuracy: 0.7315
Epoch 7/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5489 - accuracy: 0.7311
Epoch 8/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5478 - accuracy: 0.7313
Epoch 9/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5477 - accuracy: 0.7314
Epoch 10/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5469 - accuracy: 0.7329

In [108]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5552 - accuracy: 0.7312 - 499ms/epoch - 2ms/step
Loss: 0.5551505088806152, Accuracy: 0.731195330619812


In [113]:
# Define the model - deep neural net
input_features = X_train.shape[1]
hidden_node8 = 16
hidden_node7 = 16
hidden_node6 = 16

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node8, activation="relu", input_dim=input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node7, activation="relu"))
#third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node6, activation="relu"))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_28 (Dense)            (None, 16)                704       
                                                                 
 dense_29 (Dense)            (None, 16)                272       
                                                                 
 dense_30 (Dense)            (None, 16)                272       
                                                                 
 dense_31 (Dense)            (None, 1)                 17        
                                                                 
Total params: 1265 (4.94 KB)
Trainable params: 1265 (4.94 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [114]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [115]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 4s 4ms/step - loss: 0.5956 - accuracy: 0.6978
Epoch 2/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5571 - accuracy: 0.7282
Epoch 3/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5538 - accuracy: 0.7295
Epoch 4/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5518 - accuracy: 0.7303
Epoch 5/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5501 - accuracy: 0.7302
Epoch 6/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5490 - accuracy: 0.7315
Epoch 7/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5486 - accuracy: 0.7311
Epoch 8/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5482 - accuracy: 0.7303
Epoch 9/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5474 - accuracy: 0.7310
Epoch 10/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5472 - accura

In [116]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5591 - accuracy: 0.7296 - 885ms/epoch - 3ms/step
Loss: 0.5590794086456299, Accuracy: 0.7295626997947693
